# Test recall with NER and lexicon
In which we determine which method for entity candidate generation will provide better recall. Maybe they're both terrible!!

In [23]:
import pandas as pd
import data_helpers
reload(data_helpers)
from data_helpers import get_mention_entity_lists
import os
import codecs

## Load data
Let's pick data from one of the more active Facebook groups, based on total post volume.

In [9]:
!ls -haltS ../../data/facebook-maria/*.tsv

-rw-r--r-- 1 istewart6 gtperson 1.8M Oct 25 17:58 ../../data/facebook-maria/1773209126315380_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 1.3M Oct 21 19:44 ../../data/facebook-maria/296720367472342_2017-09-20_2017-10-20_facebook_comments.tsv
-rw-r--r-- 1 istewart6 gtperson 1.1M Oct 25 17:54 ../../data/facebook-maria/132392547395104_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 1.1M Oct 25 17:54 ../../data/facebook-maria/127217134598253_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 1.1M Oct 20 19:38 ../../data/facebook-maria/296720367472342_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 849K Oct 23 17:56 ../../data/facebook-maria/274834366346071_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 753K Oct 25 17:53 ../../data/facebook-maria/891721064308258_2017-09-20_2017-10-20_facebook_posts.tsv
-rw-r--r-- 1 istewart6 gtperson 647K Oct 25 17:52 ../../data/face

In [10]:
test_facebook_file = '../../data/facebook-maria/1773209126315380_2017-09-20_2017-10-20_facebook_posts.tsv'
test_facebook_df = pd.read_csv(test_facebook_file, sep='\t', index_col=0)
print('%d statuses to test'%(test_facebook_df.shape[0]))

5626 statuses to test


Let's take a sample of 500 statuses and mark the entities (if any).

In [27]:
pd.np.random.seed(123)
sample_size = 500
status_txt = test_facebook_df.loc[:, 'status_message'].dropna(inplace=False).apply(str).replace('\t', ' ').replace('\n', ' ')
status_sample = pd.np.random.choice(status_txt, size=sample_size, replace=False)
print('\n'.join(status_sample[:10]))

*PORFAVOR*** Muy PREOCUPADA***por mi Querida Tia &Tio- Carlos (Jun) Vega Y Rosa Vega- Estancias el Vijia en Guayama . Rosanna Rivera de Orlando- sabe algo porfavor contact me.
Buenas noches estoy tratando de obtener información de My familia en olimpo de la calle 6, Carmen Iris Suren Tirado Andrea Arroyo Arelis Arroyo de la calle 8, Jenniferlee Sierra Soto de la puente, si alguien tiene alguna información no importa la hora por favor envíeme un mensaje gracias
Hola me gustaría saber si alguien conoce a Yaxie Vázquez ella vive en Guayama en el residencial Villamar, no hemos podido comunicarnos y estamos preocupados si alguien sabe de ella o su familia por favor déjenme saber por este medio. Gracias de antemano mano.
I finally spoke to my Grandparents In Guayama from Carite they are ok. Thank God but they say Everyone up there lost alot but they are all ok.The Service for Claro is working at the moment so try calling your family members
Ya supe que uno de mis hermanos y sobrinos estan bi

In [47]:
# write to file for separate annotation
group_id = os.path.basename(test_facebook_file).split('_')[0]
data_dir = os.path.dirname(test_facebook_file)
sample_file = os.path.join(data_dir, '%s_post_sample_txt.txt'%(group_id))
with codecs.open(sample_file, 'w', encoding='utf-8') as sample_file_out:
    for l in status_sample:
        l_str = '%s\n'%(l).decode('utf-8')
        sample_file_out.write(l_str)

Note: for sanity we only label the first 10 entities in each status because some of these statuses are TOO LONG.

In [52]:
# 250 annotated lines later... (also includes empty lines)
import re
annotate_matcher = re.compile('(?<=<<)[\w\s\,\.]+(?=>>)')
sample_annotated_file = os.path.join(data_dir, '%s_post_sample_annotated.txt'%(group_id))
final_line = 249
sample_annotated_txt = [l.strip() for i, l in enumerate(codecs.open(sample_annotated_file)) if i < final_line]
sample_annotated_txt_mentions = [annotate_matcher.findall(l) for l in sample_annotated_txt]

In [53]:
from collections import Counter
annotation_counts = Counter(reduce(lambda x,y: x+y, sample_annotated_txt_mentions))
top_k = 10
print(annotation_counts.most_common(top_k))

[('Guayama', 41), ('guayama', 16), ('Puerto Rico', 12), ('PR', 10), ('San Juan', 8), ('olimpo', 4), ('Hato Rey', 4), ('Arroyo', 4), ('Patillas', 4), ('Estados Unidos', 4)]


As expected, the most common entities annotated are the municipality of the group (`Guayama`) and the country (`Puerto Rico`).

## Load lexicon

In [ ]:
import data_helpers
reload(data_helpers)
from data_helpers import get_mention_entity_lists
mention_entity_lists = get_mention_entity_lists(verbose=True)

processed 0 lines
processed 1000000 lines
processed 2000000 lines
processed 3000000 lines
processed 4000000 lines
processed 5000000 lines
processed 6000000 lines
processed 7000000 lines
processed 8000000 lines
processed 9000000 lines
processed 10000000 lines
processed 11000000 lines
processed 12000000 lines
processed 13000000 lines
processed 14000000 lines
processed 15000000 lines
processed 16000000 lines
processed 17000000 lines
processed 18000000 lines
processed 19000000 lines
processed 20000000 lines
processed 21000000 lines
processed 22000000 lines
processed 23000000 lines
processed 24000000 lines
processed 25000000 lines
processed 26000000 lines
processed 27000000 lines
processed 28000000 lines
processed 29000000 lines
processed 30000000 lines
processed 31000000 lines
processed 32000000 lines
processed 33000000 lines
processed 34000000 lines
processed 35000000 lines
processed 36000000 lines
processed 37000000 lines
processed 38000000 lines
processed 39000000 lines
processed 400000

In [66]:
print(len(mention_entity_lists))

77000718


In [67]:
# need inverse to get entity-mention and then restrict to location entities
from collections import defaultdict
entity_mention_lists = defaultdict(list)
ctr = 0
for m, e_list in mention_entity_lists.iteritems():
    for e in e_list:
        entity_mention_lists[e].append(m)
    ctr += 1
    if(ctr % 1000000 == 0):
        print('processed %d mention strings'%(ctr))

processed 1000000 mention strings
processed 2000000 mention strings
processed 3000000 mention strings
processed 4000000 mention strings
processed 5000000 mention strings
processed 6000000 mention strings
processed 7000000 mention strings
processed 8000000 mention strings
processed 9000000 mention strings
processed 10000000 mention strings
processed 11000000 mention strings
processed 12000000 mention strings
processed 13000000 mention strings
processed 14000000 mention strings
processed 15000000 mention strings
processed 16000000 mention strings
processed 17000000 mention strings
processed 18000000 mention strings
processed 19000000 mention strings
processed 20000000 mention strings
processed 21000000 mention strings
processed 22000000 mention strings
processed 23000000 mention strings
processed 24000000 mention strings
processed 25000000 mention strings
processed 26000000 mention strings
processed 27000000 mention strings
processed 28000000 mention strings
processed 29000000 mention st

In [70]:
# get location strings
import gzip
location_entities = [l.strip() for l in gzip.open('/hg190/corpora/crosswikis-data.tar.bz2/location_urls.gz', 'r')]

In [75]:
location_mention_lists = {l : entity_mention_lists[l] for l in location_entities}

In [83]:
# for all locations without mentinos, add the entity name as default
for loc, mentions in location_mention_lists.iteritems():
    if(len(mentions) == 0):
        new_mentions = [loc.replace('_', ' ')]
        location_mention_lists[loc] = new_mentions

In [106]:
# now combine all the mentions
location_mention_lexicon = set()
for i, v in enumerate(location_mention_lists.values()):
    location_mention_lexicon.update(v) 
    if(i % 100000 == 0):
        print('processed %d mention lists'%(i))
print('%d location mentions'%(len(location_mention_lexicon)))

processed 0 mention lists
processed 100000 mention lists
processed 200000 mention lists
processed 300000 mention lists
processed 400000 mention lists
processed 500000 mention lists
processed 600000 mention lists
processed 700000 mention lists
processed 800000 mention lists
10721949 location mentions


In [107]:
# write to file
location_mention_lexicon_out_file = '/hg190/corpora/crosswikis-data.tar.bz2/location_mention_lexicon.gz'
with gzip.open(location_mention_lexicon_out_file, 'w') as out_file:
    for l in sorted(location_mention_lexicon):
        out_file.write('%s\n'%(l))

## Test recall
We test the ability of (1) NER and (2) location mention lexicon to extract the mention candidates.

First we need to get tags for both methods.

### Get NER tags

In [90]:
from nltk.tag.stanford import StanfordNERTagger
stanford_dir = 'stanford-ner-2017-06-09'
jar_file = os.path.join(stanford_dir, 'stanford-ner.jar')
model_file = os.path.join(stanford_dir, 'classifiers/spanish.ancora.distsim.s512.crf.ser.gz')
tagger = StanfordNERTagger(model_filename=model_file, path_to_jar=jar_file)

/hg190/istewart6/crisis_language/lib/python2.7/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [93]:
from nltk.tokenize.casual import TweetTokenizer
tokenizer = TweetTokenizer()
tag_txt = lambda x: tagger.tag(tokenizer.tokenize(x))
status_sample_ner_tagged = map(tag_txt, status_sample)

In [95]:
# fix format
status_sample_ner_tagged = map(lambda x: ' '.join(['%s/%s'%(y) for y in x]), status_sample_ner_tagged)

In [97]:
print('\n'.join(status_sample_ner_tagged[:10]))

*/O PORFAVOR/ORG */O */O */O Muy/O PREOCUPADA/O */O */O */O por/O mi/O Querida/ORG Tia/ORG &/ORG Tio/ORG -/ORG Carlos/ORG (/ORG Jun/ORG )/ORG Vega/ORG Y/O Rosa/OTROS Vega/OTROS -/O Estancias/O el/O Vijia/ORG en/O Guayama/LUG ./O Rosanna/PERS Rivera/PERS de/O Orlando/OTROS -/O sabe/O algo/O porfavor/O contact/O me/O ./O
Buenas/O noches/O estoy/O tratando/O de/O obtener/O información/O de/O My/O familia/O en/O olimpo/O de/O la/O calle/O 6/O ,/O Carmen/PERS Iris/PERS Suren/PERS Tirado/O Andrea/PERS Arroyo/PERS Arelis/PERS Arroyo/PERS de/O la/O calle/O 8/O ,/O Jenniferlee/PERS Sierra/PERS Soto/PERS de/O la/O puente/O ,/O si/O alguien/O tiene/O alguna/O información/O no/O importa/O la/O hora/O por/O favor/O envíeme/O un/O mensaje/O gracias/O
Hola/O me/O gustaría/O saber/O si/O alguien/O conoce/O a/O Yaxie/PERS Vázquez/PERS ella/O vive/O en/O Guayama/LUG en/O el/O residencial/O Villamar/PERS ,/O no/O hemos/O podido/O comunicarnos/O y/O estamos/O preocupados/O si/O alguien/O sabe/O de/O ella/

In [8]:
from data_helpers import collect_entities_from_txt
from collections import Counter
status_sample_entity_list = map(collect_entities_from_txt, status_sample_ner_tagged)
status_sample_entity_counts = Counter(reduce(lambda x,y: x+y, status_sample_entity_list))
print(status_sample_entity_counts.most_common(10))

[('Guayama', 84), ('Carr', 38), ('Dios', 30), ('Puerto_Rico', 27), ('PR', 24), ('Urb', 20), ('Gracias', 17), ('Maria', 12), ('San_Juan', 11), ('Olimpo', 9)]


Write to file!!

In [108]:
ner_out_file = os.path.join('../../data/facebook-maria/', '%s_post_sample_ner.txt'%(group_id))
with codecs.open(ner_out_file, 'w', encoding='utf-8') as ner_out:
    for l in status_sample_ner_tagged:
        ner_out.write('%s\n'%(l))

### Get lexicon tags

Now lexicon. We'll use all n-grams on range 1-4 and look for the overlap between the lexicon and n-grams.

In [3]:
# reload corpus because dead kernel
import codecs
status_sample_file_name = '../../data/facebook-maria/1773209126315380_post_sample_txt.txt'
status_sample = [l.strip() for l in codecs.open(status_sample_file_name, 'r', encoding='utf-8')]

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.casual import TweetTokenizer
min_df = 1
max_df = 0.3
ngram_range = (1,4)
tokenizer = TweetTokenizer()
cv = CountVectorizer(min_df=1, max_df=max_df, ngram_range=ngram_range, tokenizer=tokenizer.tokenize)
status_sample_dtm = cv.fit_transform(status_sample)

In [35]:
print(status_sample_dtm.shape)

(500, 60601)


In [100]:
# reload location mention strings because dead kernel
import gzip
location_mention_lexicon = [l.strip() for l in gzip.open('/hg190/corpora/crosswikis-data.tar.bz2/location_mention_lexicon.gz')]
location_entities = [l.strip().replace('_', ' ') for l in gzip.open('/hg190/corpora/crosswikis-data.tar.bz2/location_urls.gz', 'r')]
location_mention_lexicon += location_entities
location_mention_lexicon = sorted(set(location_mention_lexicon))


In [101]:
# get lexicon overlap
ivoc = {v:k for k,v in cv.vocabulary_.iteritems()}
status_sample_lexicon = sorted(set(cv.vocabulary_.keys()) & set(location_mention_lexicon))
print('%d overlap words'%(len(status_sample_lexicon)))
print(status_sample_lexicon[100:200])

2057 overlap words
[u'615', u'624', u'65', u'694', u'696', u'700', u'75', u'77', u'78', u'80', u'80 %', u'81', u'824', u'829', u'830', u'833', u'84', u'842', u'843', u'845', u'852', u'88', u'90', u'91', u'938', u'95', u'950', u'97', u'975', u'99', u'999', u'9am', u':', u':p', u'<3', u'a casa', u'a couple', u'a ir', u'a la', u'a little', u'a message', u'a partir', u'a section', u'aa', u'aaa', u'abajo', u'able', u'about', u'about it', u'about them', u'above', u'abrir', u'across', u'action', u'ada', u'address', u'addresses', u'aeropuerto', u'aeropuerto de', u'aeropuerto internacional', u'aeropuertos', u'after', u'again', u'age', u'agency', u'ago', u'agua', u'aguadilla', u'aguadilla airport', u'aguas', u'aguas buenas', u'ah', u'aibonito', u'aircraft', u'aire', u'airlines', u'airport', u'aka', u'al', u'al parque', u'al pueblo', u'ala', u'alcalde', u'alejandrino', u'alerta', u'alex', u'algo', u'alguen', u'alguna', u'alguna otra', u'algunas', u'algunos', u'alida', u'alimentos', u'all', u'all 

In [103]:
status_sample_lexicon_idx = set([cv.vocabulary_[v] for v in status_sample_lexicon])

In [104]:
import numpy as np
valid_indices = [set(row.nonzero()[1]) & set(status_sample_lexicon_idx) for row in status_sample_dtm]
# convert to vocab
status_sample_lexicon_entities = [[ivoc[i] for i in idx] for idx in valid_indices]
print(status_sample_lexicon_entities[0:10])

[[u'vega', u'carlos', u'tio', u'el', u'orlando', u'tia', u'sabe', u'rivera', u'guayama', u'contact', u'rosa', u'querida', u'*', u'me', u'muy', u'estancias', u'algo'], [u'iris', u'puente', u'la calle', u'un', u'familia', u'la puente', u'hora', u'calle', u'sierra', u'no', u'favor', u'mensaje', u'de la calle', u'olimpo', u'alguna', u'gracias', u'soto', u'tiene', u'de la', u'my'], [u'villamar', u'este', u'me', u'guayama', u'hola', u'medio', u'ella o', u'familia', u'su', u'no', u'el', u'gracias', u'sabe', u'favor', u'ella'], [u'finally', u'to', u'calling', u'members', u'family', u'at', u'service', u'for', u'god', u'everyone', u'thank', u'they say', u'but', u'guayama', u'family members', u'at the moment', u'moment', u'so', u'are', u'there', u'are all', u'lost', u'in', u'is', u'ok', u'try', u'they', u'they are', u'from', u'at the', u'your', u'alot', u'say', u'up', u'working', u'my', u'up there', u'the', u'all'], [u'hacienda', u'alma', u'gonzalez', u'sabe', u'santiago', u'sin', u'54', u'wanda'

### Test mention extraction scores

In [62]:
# reload NER tags because dead kernel
import codecs
from data_helpers import collect_entities_from_txt
status_sample_ner_tagged = [l.strip() for l in codecs.open('../../data/facebook-maria/1773209126315380_post_sample_ner.txt')]
# collect entities
status_sample_ner_entity_list = map(collect_entities_from_txt, status_sample_ner_tagged)

In [132]:
from __future__ import division
from itertools import izip
def test_precision_recall(tagged_entity_list, gold_entity_list):
    """
    Compute the precision and recall of entity recognition 
    based on the number of entities recovered out of
    all possible entities.
    
    Parameters:
    -----------
    tagged_entity_list : list
    gold_entity_list : list
    
    Returns:
    --------
    all_false_tags : set
    all_missed_tags : set
    precision : float
    recall : float
    """
    TP = 0
    FP = 0
    FN = 0
    all_false_tags = []
    all_missed_tags = []
    for tagged, gold in izip(tagged_entity_list, gold_entity_list):
        true_tags = set(tagged) & set(gold)
        false_tags = set(tagged) - set(gold)
        missed_tags = set(gold) - set(tagged)
        TP += len(true_tags)
        FP += len(false_tags)
        FN += len(missed_tags)
        all_false_tags.append(list(false_tags))
        all_missed_tags.append(list(missed_tags))
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return all_false_tags, all_missed_tags, precision, recall

In [55]:
# load gold entity lists
import re, os, codecs
annotate_matcher = re.compile('(?<=<<)[\w\s\,\.]+(?=>>)')
sample_annotated_file = '../../data/facebook-maria/1773209126315380_post_sample_annotated.txt'
final_line = 249
sample_annotated_txt = [l.strip() for i, l in enumerate(codecs.open(sample_annotated_file)) if i < final_line]
sample_annotated_txt_mentions = [annotate_matcher.findall(l) for l in sample_annotated_txt]
N = len(sample_annotated_txt_mentions)

In [133]:
ner_false_tags, ner_missed_tags, ner_precision, ner_recall = test_precision_recall(status_sample_ner_entity_list[:N], sample_annotated_txt_mentions)
print('NER precision = %.3f, recall = %.3f'%(ner_precision, ner_recall))

NER precision = 0.106, recall = 0.201


Wow! That recall is terrible. This might have to do with the noise of the data hurting recall.

In [105]:
_, _, lexicon_precision, lexicon_recall = test_precision_recall(status_sample_lexicon_entities[:N], sample_annotated_txt_mentions)
print('lexicon precision = %.3f, recall = %.3f'%(lexicon_precision, lexicon_recall))

lexicon precision = 0.008, recall = 0.074


Precision is bad, which makes sense because of all the common words that snuck into the lexicon. It also has worse recall, maybe because we didn't collect the mentions correctly or because we should have lowercased everything first.

Let's do that as a sanity check!!

In [113]:
location_mention_lower = map(str.lower, location_mention_lexicon)

In [120]:
min_df = 1
max_df = 0.3
ngram_range = (1,4)
tokenizer = TweetTokenizer()
cv = CountVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range, tokenizer=tokenizer.tokenize, lowercase=True)
status_sample_dtm = cv.fit_transform(status_sample)
ivoc = {v:k for k,v in cv.vocabulary_.iteritems()}
status_sample_lexicon = sorted(set(cv.vocabulary_.keys()) & set(location_mention_lower))
status_sample_lexicon_idx = set([cv.vocabulary_[v] for v in status_sample_lexicon])
valid_indices = [set(row.nonzero()[1]) & set(status_sample_lexicon_idx) for row in status_sample_dtm]
# convert to vocab
status_sample_lexicon_entities_lower = [[ivoc[i] for i in idx] for idx in valid_indices]
print(status_sample_lexicon_entities_lower[0:10])

[[u'vega', u'carlos', u'tio', u'el', u'orlando', u'rosanna', u'sabe', u'rivera', u'guayama', u'contact', u'rosa', u'querida', u'*', u'de orlando', u'tia', u'me', u'muy', u'estancias', u'jun', u'algo'], [u'un', u'familia', u'calle', u'soto', u'carmen', u'hora', u'arroyo', u'no', u'buenas', u'tiene', u'gracias', u'iris', u'buenas noches', u'sierra', u'mensaje', u'alguna', u'de la', u'favor', u'la calle', u'la puente', u'andrea', u'de la calle', u'puente', u'olimpo', u'my'], [u'villamar', u'este', u'me', u'guayama', u'hola', u'medio', u'ella o', u'familia', u'su', u'no', u'favor', u'mano', u'el', u'gracias', u'sabe', u'vive', u'ella'], [u'finally', u'to', u'calling', u'members', u'family', u'at', u'service', u'for', u'god', u'everyone', u'thank', u'they say', u'but', u'guayama', u'family members', u'at the moment', u'moment', u'so', u'are', u'there', u'are all', u'lost', u'in', u'is', u'ok', u'try', u'they', u'they are', u'from', u'at the', u'your', u'claro', u'alot', u'say', u'up', u'wor

In [134]:
sample_annotated_txt_mentions_lower = map(lambda x: map(str.lower, x), sample_annotated_txt_mentions)
lexicon_false_tags, lexicon_missed_tags, lexicon_precision, lexicon_recall = test_precision_recall(status_sample_lexicon_entities_lower[:N], sample_annotated_txt_mentions_lower)
print('lexicon precision = %.3f, recall = %.3f'%(lexicon_precision, lexicon_recall))

lexicon precision = 0.037, recall = 0.421


Much better recall! I guess the lexicon isn't as garbage as I thought. 

What is the lexicon getting that NER is missing? Maybe we have to combine both for 

**MAXIMUM OVERDRIVE**

but let's get an idea of the errors first.

In [135]:
ner_missed_tags_list = reduce(lambda x,y: x+y, ner_missed_tags)
lexicon_missed_tags_list = reduce(lambda x,y: x+y, lexicon_missed_tags)
ner_missed_tags_set = set(map(str.lower, ner_missed_tags_list))
lexicon_missed_tags_set = set(lexicon_missed_tags_list)
joint_missed_tags = ner_missed_tags_set & lexicon_missed_tags_set
print(len(joint_missed_tags))
print(sorted(joint_missed_tags))

228
['19 calle', '65th infantry', 'adjuntas', 'aguirre', 'aguirre, pr', 'anasco', 'apartamento santa rosa', 'areciblo pueblo', 'atenas college', 'av. franklin delano roosevelt', 'ave barbosa', 'ave baxter', 'ave domenech', 'ave hostos', 'ave lomas verdes', 'ave los dominicos', 'ave los patriotas', 'ave magnolia', 'ave maruca', 'ave pedro albizu campos', 'ave ponce de leon', 'ave ramon l rodriguez', 'ave ramon luis rivera', 'ave roberto h todd', 'ave. de diego', 'ave. irlanda height', 'ave. los veteranos', 'ave. montecarlo', 'ave. santa juanita', 'bairoa', 'barceloneta', 'barreras', 'barriada santa ana', 'barrio blondet', 'barrio brandeis', 'barrio branderi', 'barrio corazon', 'barrio de carite', 'barrio del olvido', 'barrio linea capo', 'barrio olimpo', 'barrio pueblo', 'barrio singapur', 'barrios obrero', 'bayamon gardens', 'bda blondet', 'bda santa ana', 'blonde', 'blondet', 'blvd dr.', 'bo candelaria', 'bo. carite', 'bomberos de guayama', 'boqueron', 'cabo rojo', 'calimano', 'calle 

OK! So both NER and the lexicon missed things like:

- obscure neighborhoods (`barrio blondet`)
- roads (`calle 46`, `ave. montecarlo`)
- buildings (`apartamento santa rosa`)
- long names (`aguirre, pr`)
- respellings (`pto rico`)

Why are obvious matches like `guayama`, `san antonio` are not getting caught when they are actually in the lexicon!!

In [146]:
test_mention_strings = ['guayama', 'san antonio']
for s in test_mention_strings:
    test_word_missed_status_idx = [i for i,t in enumerate(lexicon_missed_tags) if s in t]
    status_sample_lower = map(lambda x: x.lower(), status_sample)
    test_word_missed_status = [status_sample_lower[i] for i in test_word_missed_status_idx]
    print('test mention string "%s" not detected in the following statuses'%(s))
    print(test_word_missed_status)

test mention string "guayama" not detected in the following statuses
[u'palmas bajas-guayama video gracias a la dama r ivelisse ortiz']
test mention string "san antonio" not detected in the following statuses
[u'last updated oct. 1, 2017, 7 p.m. etoct. 1, 2017, 7 p.m. ethurricane mariacaribbeanmonday will see more mail operations across puerto rico and the virgin islands as inbound mail from the mainland continues to be transported and processed across the caribbean.   postal employees are in full force providing mail and postal retail services for customers in postal facilities that were not significantly damaged by hurricane maria.if an office is damaged, postal officials are designating nearby locations for customers to receive mail. in some cases in both puerto rice and the virgin islands, letter carriers have begun delivery where safe.puerto ricopartial delivery has started today where safe to do so for the following locations:65th infantry, station, 100 calle alondra, 00923.bayam

OK! So the errors are probably coming from tokenization errors: 

The string `palmas bajas-guayama` should be split into `[palmas, bajas, guayama]` but is instead split into `[palmas, bajas-guayama]` because there is no hyphen splitting rule.

In [111]:
joint_false_tags = set(ner_false_tags) & set(lexicon_false_tags)
print(len(joint_false_tags))
print(sorted(joint_false_tags))

7
['am', 'damaged', 'facebook', 'fam', 'government', 'papi', 'salvacion']


The shared false tags are mostly common words that shouldn't be in the lexicon.

The other false tags must diverse significantly for the NER and lexicon methods.

In [151]:
ner_only_missed_tags = set(ner_missed_tags_list) - set(lexicon_missed_tags_list)
lexicon_only_missed_tags = set(lexicon_missed_tags_list) - set(ner_missed_tags_list)
print('NER only missed tags = %d'%(len(ner_only_missed_tags)))
print(sorted(ner_only_missed_tags)[:100])
print('lexicon only missed tags = %d'%(len(lexicon_only_missed_tags)))
print(sorted(lexicon_only_missed_tags)[:100])

NER only missed tags = 272
['19 Calle', '65th Infantry', 'Adjuntas', 'Aguadilla Airport', 'Aguas Buenas', 'Aguirre', 'Aguirre, PR', 'Aibonito', 'Algarrobo', 'Anasco', 'Areciblo Pueblo', 'Arecibo', 'Arroyo', 'Atenas College', 'Av. Franklin Delano Roosevelt', 'Ave Barbosa', 'Ave Baxter', 'Ave Domenech', 'Ave Hostos', 'Ave Lomas Verdes', 'Ave Los Dominicos', 'Ave Los Patriotas', 'Ave Magnolia', 'Ave Maruca', 'Ave Pedro Albizu Campos', 'Ave Ponce de Leon', 'Ave Ramon L Rodriguez', 'Ave Ramon Luis Rivera', 'Ave Roberto H Todd', 'Ave. De Diego', 'Ave. Irlanda Height', 'Ave. Los Veteranos', 'Ave. Montecarlo', 'Ave. Santa Juanita', 'BARRIO CORAZON', 'Bairoa', 'Barceloneta', 'Barreras', 'Barriada Santa Ana', 'Barrio Brandeis', 'Barrio Corazon', 'Barrio Linea Capo', 'Barrio Olimpo', 'Barrio Pueblo', 'Barrio Singapur', 'Barrio corazon', 'Barrios Obrero', 'Bayamon', 'Bayamon Gardens', 'Bda Santa Ana', 'Blondet', 'Blvd Dr.', 'Bo candelaria', 'Bo. Carite', 'Bomberos de Guayama', 'Boqueron', 'Borinqu

In [ ]:
print(lexicon_false_tags)

Outside of the very obscure mentions (`calle igualdad`, `apartamento santa rosa`), there are words in the lexicon-only missed tags that should not have been missed:

`aguirre`, `ceiba airport`, `cidra`

In [155]:
print('cidra' in location_mention_lower)

True


In [156]:
test_mention_strings = ['aguirre', 'ceiba airport', 'cidra']
for s in test_mention_strings:
    test_word_missed_status_idx = [i for i,t in enumerate(lexicon_missed_tags) if s in t]
    status_sample_lower = map(lambda x: x.lower(), status_sample)
    test_word_missed_status = [status_sample_lower[i].replace(s, '**%s**'%(s)) for i in test_word_missed_status_idx]
    print('test mention string "%s" not detected the following statuses'%(s))
    print(test_word_missed_status)

test mention string "aguirre" not detected the following statuses
[u'last updated oct. 1, 2017, 7 p.m. etoct. 1, 2017, 7 p.m. ethurricane mariacaribbeanmonday will see more mail operations across puerto rico and the virgin islands as inbound mail from the mainland continues to be transported and processed across the caribbean.   postal employees are in full force providing mail and postal retail services for customers in postal facilities that were not significantly damaged by hurricane maria.if an office is damaged, postal officials are designating nearby locations for customers to receive mail. in some cases in both puerto rice and the virgin islands, letter carriers have begun delivery where safe.puerto ricopartial delivery has started today where safe to do so for the following locations:65th infantry, station, 100 calle alondra, 00923.bayamon, branch, 100 ave ramon l rodriguez, 00956.cupey, station, 369 calle san claudio frnt, 00926.hato rey, station, 361 calle juan calaf, 00917.o

More tokenization errors. This suggests that we should build a custom function using TweetTokenizer plus a few extra rules involving slashes and dashes.

## Entity linking baseline: wiki links
We need a baseline for entity linking. We should redo the annotation and add the actual Wiki links where possible, then use this to determine the precision/recall (@k) of the entity candidates generated by the CrossWikis lexicon. 

...many hours later...

First let's count how many of the annotated entities actually have Wiki pages.

In [172]:
annotate_matcher = re.compile('<<([\w\s\,\.]+)>>\[\[(\S*)\]\]')
test = """
*PORFAVOR*** Muy PREOCUPADA***por mi Querida Tia &Tio- Carlos (Jun) Vega Y Rosa Vega- Estancias el Vijia en <<Guayama>>[[https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico]] . Rosanna Rivera de Orlando- sabe algo porfavor contact me.
"""
print(annotate_matcher.findall(test))

[('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico')]


In [175]:
import re
annotate_matcher = re.compile('<<([\w\s\,\.]+)>>\[\[(\S+)\]\]')
data_dir = '../../data/facebook-maria/'
group_id = 1773209126315380
sample_annotated_file = os.path.join(data_dir, '%s_post_sample_annotated_wiki.txt'%(group_id))
final_line = 249
sample_annotated_wiki_txt = [l.strip() for i, l in enumerate(codecs.open(sample_annotated_file)) if i < final_line]
sample_annotated_wiki_txt_mentions = [annotate_matcher.findall(l) for l in sample_annotated_wiki_txt]

In [176]:
print(sample_annotated_wiki_txt_mentions[:10])

[[('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico')], [('olimpo', 'NIL'), ('calle 6', 'NIL'), ('calle 8', 'NIL')], [('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico'), ('residencial Villamar', 'NIL')], [('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico'), ('Carite', 'NIL')], [('calle 54', 'NIL')], [('Guayma', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico'), ('villa Rosa', 'NIL'), ('calle 2', 'NIL')], [('Guayama URB', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico'), ('Hacienda AS', 'NIL'), ('19 Calle', 'NIL')], [], [], [('barrio mosquito', 'https://es.wikipedia.org/wiki/Mosquito_(Vieques)')]]


In [178]:
wiki_mentions_collapsed = reduce(lambda x,y: x+y, sample_annotated_wiki_txt_mentions)
print('%d mentions total'%(len(wiki_mentions_collapsed)))
non_nil_wiki_mentions = filter(lambda x: x[1]!='NIL', wiki_mentions_collapsed)
non_nil_wiki_mention_pct = len(non_nil_wiki_mentions) / len(wiki_mentions_collapsed) * 100
print('%.3f%% valid mentions'%(non_nil_wiki_mention_pct))

328 mentions total
53.354% valid mentions


OK! About half the mentions are valid. What are some of the invalid mentions?

In [179]:
nil_wiki_mentions = filter(lambda x: x[1]=='NIL', wiki_mentions_collapsed)
print(nil_wiki_mentions[:20])

[('olimpo', 'NIL'), ('calle 6', 'NIL'), ('calle 8', 'NIL'), ('residencial Villamar', 'NIL'), ('Carite', 'NIL'), ('calle 54', 'NIL'), ('villa Rosa', 'NIL'), ('calle 2', 'NIL'), ('Hacienda AS', 'NIL'), ('19 Calle', 'NIL'), ('Vives', 'NIL'), ('Los Barros', 'NIL'), ('la carr 15', 'NIL'), ('La Gran Parada', 'NIL'), ('villa Rosa', 'NIL'), ('calle 2', 'NIL'), ('las mareas', 'NIL'), ('Jardines de Guamani', 'NIL'), ('Cimarrona', 'NIL'), ('Jardines de guamani', 'NIL')]


Mostly obscure neighborhoods (`olimpo`) and roads (`calle 2`). Maybe I shouldn't have marked those.

In [180]:
non_nil_wiki_mentions = map(lambda x: filter(lambda y: y[1]!='NIL', x), sample_annotated_wiki_txt_mentions)
print(non_nil_wiki_mentions[:5])

[[('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico')], [], [('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico')], [('Guayama', 'https://en.wikipedia.org/wiki/Guayama,_Puerto_Rico')], []]


Let's see what recall/precision are like for NER and lexicon with just the Wiki mentions.

In [185]:
# just need the text for accuracy
non_nil_wiki_mentions_txt = map(lambda x: map(lambda y: y[0], x), non_nil_wiki_mentions)
non_nil_wiki_mentions_txt_lower = map(lambda x: map(lambda y: y.lower(), x), non_nil_wiki_mentions_txt)
print(non_nil_wiki_mentions_txt_lower[:10])

[['guayama'], [], ['guayama'], ['guayama'], [], ['guayma'], ['guayama urb'], [], [], ['barrio mosquito']]


In [186]:
ner_false_tags, ner_missed_tags, ner_precision, ner_recall = test_precision_recall(status_sample_ner_entity_list[:N], non_nil_wiki_mentions_txt)
lexicon_false_tags, lexicon_missed_tags, lexicon_precision, lexicon_recall = test_precision_recall(status_sample_lexicon_entities_lower[:N], non_nil_wiki_mentions_txt_lower)

In [188]:
print('on Wiki annotations:')
print('NER precision=%.3f, recall=%.3f'%(ner_precision, ner_recall))
print('lexicon precision=%.3f, recall=%.3f'%(lexicon_precision, lexicon_recall))

on Wiki annotations:
NER precision=0.072, recall=0.389
lexicon precision=0.025, recall=0.825


Nice!! So lexicon gets pretty much every single Wiki-linked entity. Probably because half of them are `Guayama` BUT STILL.

### Actual entity linking

For each anchor string we've computed the entity with the maximum conditional probability, based on the CrossWikis probabilities:

$$argmax_{e} \: P(s | e)$$

So now let's compute the highest possible precision: assuming that the Wiki-searchable entities were perfectly extracted, what percentage of the entities are correctly disambiguated using the entity with the max probability?

In [ ]:
import pandas as pd
from bz2 import BZ2File
# LIMIT TO DATASET to avoid memory sadness
full_mention_set = set(reduce(lambda x,y: x+y, status_sample_ner_entity_list[:N]))
full_mention_set_lower = set(reduce(lambda x,y: x+y, status_sample_lexicon_entities_lower[:N]))
prob_file = '/hg190/corpora/crosswikis-data.tar.bz2/max_prob_entities.bz2'
max_prob_entities = []
max_prob_entities_lower = []
for i, l in enumerate(BZ2File(prob_file, 'r')):
    # skip header line
    if(i > 0):
        mention, prob, entity = l.strip().split('\t')
        prob = float(prob)
        mention_lower = mention.lower()
        if(mention in full_mention_set):
            max_prob_entities.append([mention, prob, entity])
        elif(mention_lower in full_mention_set_lower):
            max_prob_entities_lower.append([mention_lower, prob, entity])
col_names = ['mention', 'prob', 'entity']
max_prob_entities = pd.DataFrame(max_prob_entities, columns=col_names)
max_prob_entities_lower = pd.DataFrame(max_prob_entities_lower, columns=col_names)
max_prob_entities.set_index('mention', inplace=True)
max_prob_entities_lower.set_index('mention', inplace=True)
# DON'T USE PANDAS it kills the memory ;_;
# max_probs = pd.read_csv(prob_file, sep='\t', compression='bz2')
# also convert to lowercase because that might work better
# compute max prob over lowercase
# max_probs_lower_idx = map(str.lower, max_probs.index)
# max_probs_lower = max_probs.copy()
# max_probs_lower.index = max_probs_lower_idx
# max_probs_lower = max_probs_lower.groupby(max_probs_lower_idx).apply(lambda x: x[x.loc[:, 'prob'] == x.loc[:, 'prob'].max()])

In [193]:
from __future__ import division
from itertools import izip
UNK = 'UNK'
def guess_most_likely_entities(wiki_mentions, max_probs):
    most_likely_entities = []
    for m_list in wiki_mentions:
        e_list = []
        for m in m_list:
            if(m in max_probs):
                e = max_probs.loc[m, 'entity']
            else:
                e = UNK
            e_list.append(e)
        most_likely_entities.append(e_list)
    return most_likely_entities

def compute_linking_precision(gold_entities, est_entities):
    tp = sum([len([1 if g==e else 0 for g, e in izip(g_list, e_list)]) for g_list,e_list in izip(gold_entities, est_entities)])
    fp = sum([len([1 if g!=e else 0 for g, e in izip(g_list, e_list)]) for g_list,e_list in izip(gold_entities, est_entities)])
    precision = tp / (tp + fp)
    return precision

In [ ]:
est_entities = guess_most_likely_entities(non_nil_wiki_mentions_txt, max_probs)
non_nil_wiki_mentions_entities = map(lambda x: map(lambda y: y[1], x), non_nil_wiki_mentions)
link_precision = compute_linking_precision(non_nil_wiki_mentions_entities, est_entities)
print('naive link precision with max probs = %.3f'%(link_precision))